In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
# This dataset is about heart disease, more specifically it's about coronary artery disease and angina. I will go through this dataset. First I want to test my Python skills. Second, I want to test some ideas with machine learning. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
HD=pd.read_csv ('../input/heart.csv')

In [ ]:
HD.shape

In [ ]:
HD.sample(3)

In [ ]:
# the dataset is very clean, which makes it easy to go next step.
HD.isnull().sum()

In [ ]:
HD.info()

In [ ]:
hd=HD.copy()

In [ ]:
# from the data, we can see that there seems no clear 'outcome' information. So test several basic features and also learn the number meanings from data description.
print('cp values are', hd['cp'].unique())
print('thal values are', hd['thal'].unique())
print('restecg values are', hd['restecg'].unique())
print('ca values are', hd['ca'].unique())

In [ ]:
# to make the column name more readable, such as 'target' means how narrow the coronary artery is.
hd=hd.rename(columns={'target': 'narrow', 'thal': 'duration', 'thalach':'max_HR', 'fbs':'high_fasting_sugar', 'exang':'exercise_angina','ca': 'coronary_num'})

In [ ]:
hd.sample(3)

In [ ]:
# Correlation test
sns.heatmap(hd.corr(),annot=True,cmap='RdYlGn') 
fig=plt.gcf()
fig.set_size_inches(10,8)
plt.show

In [ ]:
# two factors correlation
fig, ax=plt.subplots( figsize=(6,6))
sns.regplot(x="cp", y="age", data=hd,)

In [ ]:
sns.distplot(HD["ca"] )

In [ ]:
# distribution. 
fig, ax=plt.subplots(1,3, figsize=(15,5))
sns.kdeplot(hd['age'], ax=ax[0])
sns.kdeplot(hd['trestbps'], ax=ax[1])
sns.kdeplot(hd['max_HR'], ax=ax[2])

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# I set chest pain as the disease outcome
x=hd.drop(['cp'] , axis=1)
y=hd['cp']

In [ ]:
# look at which features are important for the 'Outcome'
model= RandomForestClassifier(n_estimators=100,random_state=0)
model.fit(x,y)
pd.Series(model.feature_importances_,index=x.columns).sort_values(ascending=False)

#remember that 'max_HR' has the highest score

In [ ]:
# Since we already set the 'outcome', some supervised models are considered
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [ ]:
Xtrain,Xtest, Ytrain, Ytest=train_test_split(x, y, test_size=0.2,random_state=42,stratify=y) # split data to train and test parts

In [ ]:
# evaluate the accuracies of the models
abc=[]
classifiers=['Linear Svm','Radial Svm','Logistic Regression','KNN','Decision Tree']
models=[svm.SVC(kernel='linear'),svm.SVC(kernel='rbf'),LogisticRegression(),KNeighborsClassifier(n_neighbors=3),DecisionTreeClassifier()]
for i in models:
    model = i
    model.fit(Xtrain,Ytrain)
    prediction=model.predict(Xtest)
    abc.append(metrics.accuracy_score(prediction,Ytest))

models_dataframe=pd.DataFrame(abc,index=classifiers)   
models_dataframe

In [ ]:
# Notice that the highest score is only 0.6. Based on the feature score list, to 8 features have scores above 0.5. To see whether these important features will produce more accurate prediction
hd_8=hd.loc[:, ['max_HR', 'chol', 'age', 'trestbps', 'oldpeak', 'narrow', 'exercise_angina', 'coronary_num']]
Xtrain,Xtest, Ytrain, Ytest=train_test_split(hd_8, y, test_size=0.2,random_state=42,stratify=y)
abc8=[]
classifiers=['Linear Svm','Radial Svm','Logistic Regression','KNN','Decision Tree']
models=[svm.SVC(kernel='linear'),svm.SVC(kernel='rbf'),LogisticRegression(),KNeighborsClassifier(n_neighbors=3),DecisionTreeClassifier()]
for i in models:
    model = i
    model.fit(Xtrain,Ytrain)
    prediction=model.predict(Xtest)
    abc8.append(metrics.accuracy_score(prediction,Ytest))

models_df8=pd.DataFrame(abc8,index=classifiers)   
models_df8

In [ ]:
# surprisingly, top 8 important features are less accurate than the total features. How about the top 4?
hd_4=hd.loc[:, ['max_HR', 'chol', 'age', 'trestbps']]
Xtrain,Xtest, Ytrain, Ytest=train_test_split(hd_4, y, test_size=0.2,random_state=42,stratify=y)
abc4=[]
classifiers=['Linear Svm','Radial Svm','Logistic Regression','KNN','Decision Tree']
models=[svm.SVC(kernel='linear'),svm.SVC(kernel='rbf'),LogisticRegression(),KNeighborsClassifier(n_neighbors=3),DecisionTreeClassifier()]
for i in models:
    model = i
    model.fit(Xtrain,Ytrain)
    prediction=model.predict(Xtest)
    abc4.append(metrics.accuracy_score(prediction,Ytest))

models_df4=pd.DataFrame(abc4,index=classifiers)   
models_df4

In [ ]:
# Even worse!

In [ ]:
# Maybe standarlization could help?
from sklearn.preprocessing import StandardScaler

In [ ]:
features=x[x.columns[:13]]
features_standard=StandardScaler().fit_transform(features)

In [ ]:
hd_std=pd.DataFrame(features_standard,columns=[[  'age', 'sex', 'trestbps', 'chol', 'high_fasting_sugar', 'restecg', 'max_HR','exercise_angina', 'oldpeak', 'slope', 'coronary_num','duration','narrow' ]])


In [ ]:
hd_std.head(1)

In [ ]:
# try again with standarlizaed data

In [ ]:
Xtrain,Xtest, Ytrain, Ytest=train_test_split(hd_std, y, test_size=0.2,random_state=42,stratify=y)
abc_std=[]
classifiers=['Linear Svm','Radial Svm','Logistic Regression','KNN','Decision Tree']
models=[svm.SVC(kernel='linear'),svm.SVC(kernel='rbf'),LogisticRegression(),KNeighborsClassifier(n_neighbors=3),DecisionTreeClassifier()]
for i in models:
    model = i
    model.fit(Xtrain,Ytrain)
    prediction=model.predict(Xtest)
    abc_std.append(metrics.accuracy_score(prediction,Ytest))

models_df_std=pd.DataFrame(abc_std,index=classifiers)   
models_df_std

In [ ]:
hd_8std=hd_std.loc[:, ['max_HR', 'chol', 'age', 'trestbps', 'oldpeak', 'narrow', 'exercise_angina', 'coronary_num']]
Xtrain,Xtest, Ytrain, Ytest=train_test_split(hd_8std, y, test_size=0.2,random_state=42,stratify=y)
abc_8std=[]
classifiers=['Linear Svm','Radial Svm','Logistic Regression','KNN','Decision Tree']
models=[svm.SVC(kernel='linear'),svm.SVC(kernel='rbf'),LogisticRegression(),KNeighborsClassifier(n_neighbors=3),DecisionTreeClassifier()]
for i in models:
    model = i
    model.fit(Xtrain,Ytrain)
    prediction=model.predict(Xtest)
    abc_8std.append(metrics.accuracy_score(prediction,Ytest))

models_df_8std=pd.DataFrame(abc_8std,index=classifiers)   
models_df_8std

In [ ]:
# For other models, it seems improve a little. For the LInear Svm, not better.
# So look back at the data, whether we miss something?
# For 'cp' column, it records chest pain type. Number 3 mean no chest pain, number 0-2 means different tyoe of angina. So is it possible that this kind of recording makes the problem too complicated?
# To simplify it, I will group the number0-2 together as disease positive, number 3 as disease negative

In [ ]:
hd['cp'].isin([0,1,2]).value_counts()

In [ ]:
# The dataset contain 23 ' disease negative' patient

In [ ]:
hd_mod=HD.copy()

In [ ]:
number=[0,1,2]
for col in hd.itertuples():

    if col.cp in number:
        hd_mod['cp'].replace(to_replace=col.cp, value=1, inplace=True)


In [ ]:
hd_mod['cp'].value_counts()

In [ ]:
y_mod=hd_mod['cp']

In [ ]:
x.sample(3)

In [ ]:
model= RandomForestClassifier(n_estimators=100,random_state=0)
model.fit(x,y_mod)
pd.Series(model.feature_importances_,index=x.columns).sort_values(ascending=False)

In [ ]:
# Notice that after this change, the feature with highest score became 'the rest blood pressure', instead of 'max_HR'

In [ ]:
hd_std.sample(3)

In [ ]:
Xtrain,Xtest, Ytrain, Ytest=train_test_split(hd_std, y_mod, test_size=0.2,random_state=42,stratify=y)
abc_std=[]
classifiers=['Linear Svm','Radial Svm','Logistic Regression','KNN','Decision Tree']
models=[svm.SVC(kernel='linear'),svm.SVC(kernel='rbf'),LogisticRegression(),KNeighborsClassifier(n_neighbors=3),DecisionTreeClassifier()]
for i in models:
    model = i
    model.fit(Xtrain,Ytrain)
    prediction=model.predict(Xtest)
    abc_std.append(metrics.accuracy_score(prediction,Ytest))

models_df_std=pd.DataFrame(abc_std,index=classifiers)   
models_df_std

In [ ]:
# Wow! The accuracy is more than 90%?!
# IS that too good to be true?
# we look at the percentage of so called 'disease positve' patient, after we grouped number 0-2 together

In [ ]:
280/303

In [ ]:
# the patient percentage is 92%. That means the model just keep saying the prediction is '1', it still has more than 90% to be correct...
# Let's do a test

In [ ]:
hd.head(1)

In [ ]:
model=svm.SVC(kernel='linear')
model.fit(Xtrain,Ytrain)
print(model.predict([[0.952197,
0.681005,
0.763956,
-0.256334,
2.394438,
-1.005832,
0.015443,
-0.696631,
1.087338,
-2.274579,
-0.714429,
-2.148873,
0.914529]]))

In [ ]:
# Use the row 1 data to do the test and the answer should be 3, but the prediction is 1...as I thought.
# So go back to look at the data more carefully..

In [ ]:
sns.violinplot(x=HD['cp'], y=HD['target'])

In [ ]:
# Typical angina usually have less than 50% narrow artery.

In [ ]:
sns.violinplot(x=HD['cp'], y=HD['thalach'])

In [ ]:
sns.violinplot(x=HD['cp'], y=HD['age'])

In [ ]:
# look at data with 'cp' grouped

In [ ]:
sns.violinplot(x=hd_mod['cp'], y=hd_mod['chol'])

In [ ]:
sns.violinplot(x=hd_mod['cp'], y=hd_mod['thalach'])

In [ ]:
sns.violinplot(x=hd_mod['cp'], y=hd_mod['ca'])

Let's take a look at the data from another angle. If the chest pain type is not a proper outcome feature, then how about asking" the relationship of coronary artery narrowness with other factors", because the narrow level of the coronary artery will determine the prognosis of the disease.

In [ ]:
# first check the value of 'narrow' column
hd['narrow'].value_counts()

Good! This parameter contain half-half amount of two values. Let's try it again!

In [ ]:
hd_narrow=HD.copy()

In [ ]:
hd_narrow=hd_narrow.rename(columns={'target': 'narrow','pred_attribute': 'narrow', 'thal': 'duration', 'thalach':'max_HR', 'fbs':'high_fasting_sugar', 'exang':'exercise_angina','ca': 'coronary_num'})

In [ ]:
x_narrow=hd_narrow.drop(['narrow'] , axis=1)
y_narrow=hd_narrow['narrow']

In [ ]:
model= RandomForestClassifier(n_estimators=100,random_state=0)
model.fit(x_narrow,y_narrow)
pd.Series(model.feature_importances_,index=x_narrow.columns).sort_values(ascending=False)

This time the top important values are very different with previous one that 'chest pain' was set as the outcome.

In [ ]:
Xtrain_n,Xtest_n, Ytrain_n, Ytest_n=train_test_split(x_narrow, y_narrow, test_size=0.2,random_state=42,stratify=y_narrow)
abc_narrow=[]
classifiers=['Linear Svm','Radial Svm','Logistic Regression','KNN','Decision Tree']
models=[svm.SVC(kernel='linear'),svm.SVC(kernel='rbf'),LogisticRegression(),KNeighborsClassifier(n_neighbors=3),DecisionTreeClassifier()]
for i in models:
    model = i
    model.fit(Xtrain_n,Ytrain_n)
    prediction=model.predict(Xtest_n)
    abc_narrow.append(metrics.accuracy_score(prediction,Ytest_n))

models_df_narrow=pd.DataFrame(abc_narrow,index=classifiers, columns=['Score'])   
models_df_narrow

This time, the highest score is over 0.8! That means if we set the 'artery narrowness' as the outcome, it makes the whole data predictable!
Let's try some examples and see whether the model is reliable.
first, number 284 record, the 'narrow' number should be 0....

In [ ]:
model=svm.SVC(kernel='linear')
model.fit(Xtrain_n,Ytrain_n)
print(model.predict([[61,
1,
0,
140,
207,
0,
0,
138,
1,
1.9,
2,
1,
3]]))

Bingo! Try another one. Record 113. The 'narrow' number should be 1.... even though the 'cp' number is the same as Record 284

In [ ]:
model=svm.SVC(kernel='linear')
model.fit(Xtrain_n,Ytrain_n)
print(model.predict([[43,
1,
0,
110,
211,
0,
1,
161,
0,
0.0,
2,
0,
3]]))

very nice! How about the old question: top 8 features make more accurate prediction?

In [ ]:
hd_8nar=hd_narrow.loc[:, ['max_HR', 'chol', 'age', 'trestbps', 'oldpeak', 'cp', 'duration', 'coronary_num']]
Xtrain,Xtest, Ytrain, Ytest=train_test_split(hd_8nar, y_narrow, test_size=0.2,random_state=42,stratify=y_narrow)
abc_8nar=[]
classifiers=['Linear Svm','Radial Svm','Logistic Regression','KNN','Decision Tree']
models=[svm.SVC(kernel='linear'),svm.SVC(kernel='rbf'),LogisticRegression(),KNeighborsClassifier(n_neighbors=3),DecisionTreeClassifier()]
for i in models:
    model = i
    model.fit(Xtrain,Ytrain)
    prediction=model.predict(Xtest)
    abc_8nar.append(metrics.accuracy_score(prediction,Ytest))

models_df_8nar=pd.DataFrame(abc_8nar,index=classifiers, columns=['Scores'])   
models_df_8nar

not improved at all! So more features, better prediction. Lets look at the relationships of those top features with the 'narrow' feature. 

In [ ]:
fig, ax=plt.subplots(1, 4, figsize=(25,5))
sns.violinplot(x="narrow", y="cp", data=hd_narrow, ax=ax[0])
ax[0].set_xlabel('narrow', fontsize=20)
ax[0].set_ylabel('cp', fontsize=20)
sns.violinplot(x='narrow', y='oldpeak', data=hd_narrow, ax=ax[1])
ax[1].set_xlabel('narrow', fontsize=20)
ax[1].set_ylabel('ST depression', fontsize=20)
sns.violinplot(x='narrow', y='duration', data=hd_narrow, ax=ax[2])
ax[2].set_xlabel('narrow', fontsize=20)
ax[2].set_ylabel('duration of excercise', fontsize=20)
sns.violinplot(x='narrow', y='max_HR', data=hd_narrow, ax=ax[3])
ax[3].set_xlabel('narrow', fontsize=20)
ax[3].set_ylabel('max_HR', fontsize=20)

From the graphs, we can tell the distribution differences between two groups of Narrow feature. So finally we have a reasonable perspective to read this dataset.



In [ ]:
# I want to thank Kaggle for providing the wonderful place to learn and to communicate. Many code in here are learned from other kaggle members, such as I,coder.